### **Full, Incremental Extraction, and Transformation Using simulated Hospital Admissions Data**

##### **Import necessary libraries**

In [1657]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

#### **Hospital Admissions Data Simulation**
In the following code:

- A realistic hospital admissions dataset is simulated over a 60-day period starting April 1, 2025.  
- Hospitals and patient severity levels are predefined for random assignment to each admission.  
- For each day, between 3 and 6 admissions are generated.
- Records between 180 & 360(inclusive) are expected .
- Each admission is assigned a random patient ID, hospital, severity level, and a last updated timestamp with a random hour and minute on the admission date.  
- All records are stored in a list of dictionaries.  
- The list is converted into a Pandas DataFrame for easier data handling.  
- The DataFrame is saved as a CSV file named `'hospital_admissions.csv'`.  
- A preview of the first few rows is displayed to verify the structure and content of the data.


In [1658]:
import pandas as pd
import random
from datetime import datetime, timedelta
import numpy as np

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Define hospital and severity level options
hospitals = ['General Hospital', 'City Clinic', 'Mercy Medical', 'St. Mary’s', 'County Hospital']
severity_levels = ['Low', 'Moderate', 'High', 'Critical']
genders = ['Male', 'Female', 'Other']

data = []
start_date = datetime(2025, 4, 1)

# Simulate data for 60 days
for i in range(1, 61):
    date = start_date + timedelta(days=i)
    
    # Random 3–6 admissions per day
    for _ in range(random.randint(3, 6)):
        age = random.randint(0, 100)
        gender = random.choice(genders)
        
        # Introduce missing values randomly (10% chance)
        if random.random() < 0.1:
            age = None
        if random.random() < 0.1:
            gender = None

        data.append({
            'id': random.randint(1000, 9999),
            'hospital': random.choice(hospitals),
            'admission_date': date.date().isoformat(),
            'severity': random.choice(severity_levels),
            'last_updated': (date + timedelta(
                hours=random.randint(0, 23),
                minutes=random.randint(0, 59)
            )).isoformat(),
            'age': age,
            'gender': gender
        })

# Create DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv('hospital_admissions.csv', index=False)

# Preview first few rows
df.head()


,id,hospital,admission_date,severity,last_updated,age,gender
0,2679,County Hospital,2025-04-02,Low,2025-04-02T18:27:00,3.0,Other
1,1434,County Hospital,2025-04-02,Moderate,2025-04-02T22:41:00,NaN,Male
2,5557,General Hospital,2025-04-02,Moderate,2025-04-02T22:27:00,89.0,Other
3,2674,General Hospital,2025-04-03,Critical,2025-04-03T03:22:00,35.0,Male
4,8527,County Hospital,2025-04-03,Low,2025-04-03T12:05:00,44.0,None


## Extraction

#### **Full Extraction**
In the following code:
- The entire dataset is loaded from the CSV file `'hospital_admissions.csv'` with the `'last_updated'` column parsed as datetime.  
- The number of rows and columns in the dataset is displayed separately to give a detailed overview of its dimensions.  
- The total number of rows pulled is reiterated for clarity.  
- A sample of the first few rows is printed to verify the data content and structure.



In [1659]:
import pandas as pd

# Load all rows from the CSV and parse 'last_updated' as datetime
df_full = pd.read_csv("hospital_admissions.csv", parse_dates=["last_updated"])

print(f"Number of rows: {df_full.shape[0]}")    # Rows
print(f"Number of columns: {df_full.shape[1]}") # Columns

# Show how many records were pulled
print(f"Pulled {df_full.shape[0]} rows via full extraction.")

print("Sample data:")
df_full.head()


Number of rows: 274
Number of columns: 7
Pulled 274 rows via full extraction.
Sample data:


,id,hospital,admission_date,severity,last_updated,age,gender
0,2679,County Hospital,2025-04-02,Low,2025-04-02 18:27:00,3.0,Other
1,1434,County Hospital,2025-04-02,Moderate,2025-04-02 22:41:00,NaN,Male
2,5557,General Hospital,2025-04-02,Moderate,2025-04-02 22:27:00,89.0,Other
3,2674,General Hospital,2025-04-03,Critical,2025-04-03 03:22:00,35.0,Male
4,8527,County Hospital,2025-04-03,Low,2025-04-03 12:05:00,44.0,NaN


#### **Setting Initial Last Extraction Time**
In the following code:
  
- A fixed datetime string `"2025-04-20 12:00:00"` is written into a file last_extraction.txt (separately created) to simulate the last time data was extracted.  
- This timestamp serves as a reference point for future incremental data extraction processes.


In [1660]:
# Set initial last extraction time 
with open("last_extraction.txt", "w") as f:
    f.write("2025-04-20 12:00:00") 

#### **Incremental Extraction**
In the following code:

- The last extraction timestamp is read from the file `'last_extraction.txt'` and any extra whitespace is  using strip() function.  
- The full dataset is loaded from `'hospital_admissions.csv'` with the `'last_updated'` column parsed as datetime.  
- The timestamp from the file is converted into a pandas datetime object for comparison.  
- The dataset is filtered to include only rows where the `'last_updated'` timestamp is later than the last extraction time, simulating incremental extraction.  
- The number of new or updated rows since the last extraction is displayed.  
- A sample of these new/updated records is shown for verification.


In [1661]:
import pandas as pd

# Step 1: Read the last extraction timestamp from the text file
with open("last_extraction.txt", "r") as f:
    last_extraction = f.read().strip()

# Step 2: Load the full dataset and parse 'last_updated' as datetime
df = pd.read_csv("hospital_admissions.csv", parse_dates=["last_updated"])

# Step 3: Convert the last extraction time to datetime format
last_extraction_time = pd.to_datetime(last_extraction)

# Step 4: Filter rows that were updated after the last extraction time
df_incremental = df[df['last_updated'] > last_extraction_time]

# Step 5 : Save the incremental data to a new CSV file
df_incremental.to_csv("hospital_admission_incremental.csv", index=False)


# Step : Display results
print(f"Pulled {len(df_incremental)} new/updated rows since {last_extraction}.")
df_incremental.head()


Pulled 198 new/updated rows since 2025-04-20 12:00:00.


,id,hospital,admission_date,severity,last_updated,age,gender
75,8385,City Clinic,2025-04-20,Critical,2025-04-20 18:39:00,41.0,NaN
77,5526,County Hospital,2025-04-20,Low,2025-04-20 16:12:00,33.0,Female
78,4937,St. Mary’s,2025-04-20,Critical,2025-04-20 14:50:00,10.0,Male
79,4986,Mercy Medical,2025-04-20,High,2025-04-20 15:35:00,2.0,Male
80,8434,Mercy Medical,2025-04-21,High,2025-04-21 08:14:00,54.0,Other


#### **Updating the Last Extraction Timestamp**
In this code:

- The most recent `'last_updated'` timestamp in the current dataset is identified as the new checkpoint.  
- This new checkpoint timestamp is saved to the `'last_extraction.txt'` file, overwriting the previous value.  
- A confirmation message is printed to indicate that the last extraction timestamp has been updated successfully.


In [1662]:
# Step 1: Get the latest timestamp from the data
new_checkpoint = df['last_updated'].max()

# Step 2: Save this new checkpoint to the extraction file
with open("last_extraction.txt", "w") as f:
    f.write(new_checkpoint.isoformat())

# Step 3: Confirm the update
print(f"Updated last_extraction.txt to {new_checkpoint}")


Updated last_extraction.txt to 2025-05-31 22:38:00


## Transformation

### **Full data**

#### *Data Cleaning*

In [1663]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274 entries, 0 to 273
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              274 non-null    int64         
 1   hospital        274 non-null    object        
 2   admission_date  274 non-null    object        
 3   severity        274 non-null    object        
 4   last_updated    274 non-null    datetime64[ns]
 5   age             243 non-null    float64       
 6   gender          252 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 15.1+ KB


In [1664]:
df_full.duplicated().sum()

0

___
Presence of duplicated values was checked,but not found.
___

In [1665]:
df_full.isnull().sum()

id                 0
hospital           0
admission_date     0
severity           0
last_updated       0
age               31
gender            22
dtype: int64

In [1666]:
df_full["age"]=df_full["age"].fillna(df_full["age"].mean())

In [1667]:
df_full["gender"] = df_full["gender"].fillna(df_full["gender"].mode()[0])

In [1668]:
df_full.isnull().sum()

id                0
hospital          0
admission_date    0
severity          0
last_updated      0
age               0
gender            0
dtype: int64

In [1669]:
df_full['admission_date'] = pd.to_datetime(df_full['admission_date'])
df_full['age'] = df_full['age'].astype(int)

In [1670]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274 entries, 0 to 273
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              274 non-null    int64         
 1   hospital        274 non-null    object        
 2   admission_date  274 non-null    datetime64[ns]
 3   severity        274 non-null    object        
 4   last_updated    274 non-null    datetime64[ns]
 5   age             274 non-null    int32         
 6   gender          274 non-null    object        
dtypes: datetime64[ns](2), int32(1), int64(1), object(3)
memory usage: 14.0+ KB


___  
##### Data Cleaning Summary

**What was done:**
- Missing values in the `"age"` column were filled using the mean value.
- Missing values in the `"gender"` column were filled using the mode (most frequent value).
- The data type for `"age"` was converted from float to integer.
- The data type for `"admission_date"` was converted from object (string) to datetime.

**Importance:**
-  **Ensures Data Consistency**: Filling in missing values and correcting data types prevents errors during analysis and ensures uniform data.
-  **Enables Accurate Aggregation & Filtering**: Proper data types (like datetime) allow for time-based analysis, sorting, and grouping.
-  **Improves Data Quality**: Handling missing values and formatting ensures the dataset is complete and ready for reliable analysis and modeling.
___


### *Key Restructuring*

In [1671]:
# Drop the old 'id' column 
# df_full.drop(columns='id', inplace=True)

# Add a new surrogate key as 'patient_id'
df_full.insert(0, 'patient_id', range(1, len(df_full) + 1))

df_full.drop(columns='id', inplace=True)

# Preview result
df_full.head()


,patient_id,hospital,admission_date,severity,last_updated,age,gender
0,1,County Hospital,2025-04-02,Low,2025-04-02 18:27:00,3,Other
1,2,County Hospital,2025-04-02,Moderate,2025-04-02 22:41:00,48,Male
2,3,General Hospital,2025-04-02,Moderate,2025-04-02 22:27:00,89,Other
3,4,General Hospital,2025-04-03,Critical,2025-04-03 03:22:00,35,Male
4,5,County Hospital,2025-04-03,Low,2025-04-03 12:05:00,44,Other


 ___
 ##### *Why Add a Surrogate Key (`patient_id`) and Drop the Original `id`?*

- **Ensures Uniqueness**  
  A surrogate key like `patient_id` guarantees each row has a unique identifier, which is critical for data integrity—especially if the original `id` column had duplicates or gaps.

- **Decouples from Source System**  
  Using a new surrogate key prevents dependency on the original system’s identifiers, making the dataset more flexible and portable for future use.

- **Improves Data Integration**  
  Surrogate keys simplify the process of merging and integrating with other datasets by providing a clean, consistent identifier.
  ___


### *Categorization*

In [1672]:

# Define bin edges and labels
bins = [0, 12, 19, 35, 50, 65, 80, 100]
labels = ['Child', 'Teenager', 'Young Adult', 'Middle-aged Adult', 
          'Senior Adult', 'Elderly', 'Very Elderly']

# Create a new column with categorized age groups
df_full['age_group'] = pd.cut(df_full['age'], bins=bins, labels=labels, right=True, include_lowest=True)


In [1673]:


# Group by 'age_group' and find min and max ages per group
group_ranges = df_full.groupby('age_group',observed=True)['age'].agg(['min', 'max']).reset_index()

# Format ranges nicely
group_ranges['Age Range'] = group_ranges['min'].astype(str) + '–' + group_ranges['max'].astype(str)

# Rename and reorder columns
group_ranges = group_ranges[['Age Range', 'age_group']]
group_ranges.columns = ['Age Range', 'Group Label']

# Print the result
print(group_ranges.to_string(index=False))


Age Range       Group Label
     0–12             Child
    13–19          Teenager
    20–35       Young Adult
    36–50 Middle-aged Adult
    51–65      Senior Adult
    66–80           Elderly
   81–100      Very Elderly


___
##### Importance of Categorization
- **Simplifies Analysis and Visualization**  
  Grouping continuous age data into meaningful categories (e.g., Child, Teenager, etc.) makes it easier to summarize, compare, and visualize patterns in the dataset (e.g., admission trends by age group).

- **Enables Categorical Aggregation**  
  Creating age groups allows for grouping and aggregating data (like count or severity) based on age categories, which is essential for reporting and demographic analysis in healthcare datasets.
___

In [1674]:
df_full.to_csv("transformed_full.csv", index=False)

### **Incremental Data**

#### *Data Cleaning*

In [1675]:
#Load dataset with incremental updates
df_incremental= pd.read_csv("hospital_admission_incremental.csv", parse_dates=["last_updated"])

In [1676]:
df_incremental.duplicated().sum()

0

In [1677]:
df_incremental.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              198 non-null    int64         
 1   hospital        198 non-null    object        
 2   admission_date  198 non-null    object        
 3   severity        198 non-null    object        
 4   last_updated    198 non-null    datetime64[ns]
 5   age             174 non-null    float64       
 6   gender          181 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 11.0+ KB


In [1678]:

# Safely fill missing values in 'age' using .loc
df_incremental.loc[:, "age"] = df_incremental["age"].fillna(df_incremental["age"].mean())

# Safely fill missing values in 'gender' using .loc
df_incremental.loc[:, "gender"] = df_incremental["gender"].fillna(df_incremental["gender"].mode()[0])


In [1679]:
# Inspect the final structure and data types
df_incremental.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              198 non-null    int64         
 1   hospital        198 non-null    object        
 2   admission_date  198 non-null    object        
 3   severity        198 non-null    object        
 4   last_updated    198 non-null    datetime64[ns]
 5   age             198 non-null    float64       
 6   gender          198 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 11.0+ KB


In [1680]:
df_incremental['admission_date'] = pd.to_datetime(df_incremental['admission_date'])
df_incremental['age'] = df_incremental['age'].astype(int)

In [1681]:
df_incremental.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              198 non-null    int64         
 1   hospital        198 non-null    object        
 2   admission_date  198 non-null    datetime64[ns]
 3   severity        198 non-null    object        
 4   last_updated    198 non-null    datetime64[ns]
 5   age             198 non-null    int32         
 6   gender          198 non-null    object        
dtypes: datetime64[ns](2), int32(1), int64(1), object(3)
memory usage: 10.2+ KB


### *Key Restructuring*

In [1682]:
# Add a new surrogate key as 'patient_id'
df_incremental.insert(0, 'patient_id', range(1, len(df_incremental) + 1))

df_incremental.drop(columns='id', inplace=True)

# Preview result
df_incremental.head()


,patient_id,hospital,admission_date,severity,last_updated,age,gender
0,1,City Clinic,2025-04-20,Critical,2025-04-20 18:39:00,41,Other
1,2,County Hospital,2025-04-20,Low,2025-04-20 16:12:00,33,Female
2,3,St. Mary’s,2025-04-20,Critical,2025-04-20 14:50:00,10,Male
3,4,Mercy Medical,2025-04-20,High,2025-04-20 15:35:00,2,Male
4,5,Mercy Medical,2025-04-21,High,2025-04-21 08:14:00,54,Other


### *Categorization*

In [1683]:

# Define bin edges and labels
bins = [0, 12, 19, 35, 50, 65, 80, 100]
labels = ['Child', 'Teenager', 'Young Adult', 'Middle-aged Adult', 
          'Senior Adult', 'Elderly', 'Very Elderly']

# Create a new column with categorized age groups
df_incremental['age_group'] = pd.cut(df_incremental['age'], bins=bins, labels=labels, right=True, include_lowest=True)


In [1684]:
# Group by 'age_group' and find min and max ages per group
group_ranges = df_incremental.groupby('age_group',observed=True)['age'].agg(['min', 'max']).reset_index()

# Format ranges nicely
group_ranges['Age Range'] = group_ranges['min'].astype(str) + '–' + group_ranges['max'].astype(str)

# Rename and reorder columns
group_ranges = group_ranges[['Age Range', 'age_group']]
group_ranges.columns = ['Age Range', 'Group Label']

# Print the result
print(group_ranges.to_string(index=False))


Age Range       Group Label
     0–12             Child
    13–19          Teenager
    21–35       Young Adult
    36–50 Middle-aged Adult
    53–65      Senior Adult
    66–80           Elderly
   81–100      Very Elderly
